In [1]:
import findspark
from pyspark.sql import SparkSession
import re
findspark.init()

spark = SparkSession.builder.appName("t").master("local[*]").getOrCreate()
sc = spark.sparkContext


In [53]:
import findspark
from pyspark.sql import SparkSession
import re
from flask import Flask,request,render_template, jsonify, url_for
findspark.init()

spark = SparkSession.builder.appName("Trabajo_Final").master("local[*]").getOrCreate()
sc = spark.sparkContext


dict_meta = {}
with open(r"ml-1m\movies.dat", encoding="latin-1") as file:
    lines = file.readlines()
    for line in lines:
        dat = line.split("::")
        dict_meta[dat[0]] = (dat[1], dat[2].replace("\n",""))
dict_meta_bd = sc.broadcast(dict_meta)

ratings = sc.textFile(r"ml-1m\ratings.dat") \
    .map(lambda x: x.split("::")) \
    .map(lambda x: [x[1], int(x[2]), x[0]]) \
    .mapValues(lambda x: (x,1)) \
    .reduceByKey(lambda x,y : (x[0] +y[0], x[1]+ y[1])) \
    .mapValues(lambda x : (round(x[0]/x[1],2), x[1]))

final_rdd = ratings.map(lambda x: (x[0], x[1], dict_meta_bd.value[x[0]])) \
    .map(lambda x: (x[0], x[1][0], x[1][1], x[2][0], x[2][1])) \
    .map(lambda x: (
        x[0],
        x[1],
        x[2], 
        re.sub(r'\((\d+)\)', "" ,x[3] ).lower().strip(), 
        int(re.findall(r'\((\d+)\)' , x[3])[0]) , 
        x[4].lower().split("|")))

final_rdd.take(10)

[('1193', 4.39, 1725, "one flew over the cuckoo's nest", 1975, ['drama']),
 ('3408', 3.86, 1315, 'erin brockovich', 2000, ['drama']),
 ('2804', 4.24, 1352, 'christmas story, a', 1983, ['comedy', 'drama']),
 ('595',
  3.89,
  1060,
  'beauty and the beast',
  1991,
  ['animation', "children's", 'musical']),
 ('2398', 3.96, 380, 'miracle on 34th street', 1947, ['drama']),
 ('2918', 4.12, 1473, "ferris bueller's day off", 1986, ['comedy']),
 ('2791', 3.97, 1731, 'airplane!', 1980, ['comedy']),
 ('2687', 3.67, 442, 'tarzan', 1999, ['animation', "children's"]),
 ('2018', 3.74, 589, 'bambi', 1942, ['animation', "children's"]),
 ('2797', 3.86, 1491, 'big', 1988, ['comedy', 'fantasy'])]

In [4]:
ratings = sc.textFile(r"ml-1m\ratings.dat") \
    .map(lambda x: x.split("::")) \
    .map(lambda x: [x[1], int(x[2]), x[0]]) \
    .keyBy(lambda x: x[0]) \
    .mapValues(lambda x: [x[1],1, x[2]]) \
    .reduceByKey(lambda x,y : (x[0] +y[0], x[1]+ y[1], x[2])) \
    .mapValues(lambda x : (round(x[0]/x[1],2),x[1],x[2]))

dict_meta = {}
with open(r"ml-1m\movies.dat", encoding="latin-1") as file:
    lines = file.readlines()
    for line in lines:
        dat = line.split("::")
        dict_meta[dat[0]] = (dat[1], dat[2].replace("\n",""))
dict_meta_bd = sc.broadcast(dict_meta)


dict_users = {}
with open(r"ml-1m\users.dat", encoding="latin-1") as file:
    lines = file.readlines()
    for line in lines:
        dat = line.split("::")
        dict_users[dat[0]] = (dat[1], int(dat[2]))
dict_users_bd = sc.broadcast(dict_users)


final_rdd = ratings.map(lambda x: (x[0], x[1], dict_meta_bd.value[x[0]])) \
    .map(lambda x : (x[0], x[1][0],x[1][1],x[2][0],x[2][1],x[1][2])) \
    .map(lambda x: (
        x[0],
        x[1],
        x[2], 
        re.sub(r'\((\d+)\)', "" ,x[3] ).lower().strip(), 
        int(re.findall(r'\((\d+)\)' , x[3])[0]) , 
        x[4].lower().split("|"),
        x[5])) \
    .map(lambda x: (x[0],x[1],x[2],x[3],x[4],x[5],x[6]))

In [ ]:
use_rat = sc.textFile(r"ml-1m\ratings.dat") \
    .map(lambda x: x.split("::")) \
    .map(lambda x: [x[1], int(x[2]), x[0]])

dict_meta = {}
with open(r"ml-1m\movies.dat", encoding="latin-1") as file:
    lines = file.readlines()
    for line in lines:
        dat = line.split("::")
        dict_meta[dat[0]] = (dat[1], dat[2].replace("\n",""))
dict_meta_bd = sc.broadcast(dict_meta)

dict_users = {}
with open(r"ml-1m\users.dat", encoding="latin-1") as file:
    lines = file.readlines()
    for line in lines:
        dat = line.split("::")
        dict_users[dat[0]] = (dat[1], int(dat[2]))
dict_users_bd = sc.broadcast(dict_users)

use_rat.map(lambda x : (x[0],(x[1],x[2]), dict_users_bd.value[x[-1]])).take(10)




In [34]:
use_rat = sc.textFile(r"ml-1m\ratings.dat") \
    .map(lambda x: x.split("::")) \
    .map(lambda x: [x[1], int(x[2]), x[0]]) \
    .map(lambda x : (x[0],(x[1],x[2]), dict_users_bd.value[x[-1]]))  \
    .map(lambda x : (x[0], (x[1][0], 1))) \
    .reduceByKey( lambda x, y : (x[0] + y[0], x[1]+ y[1])) \
    .mapValues(lambda x: (round(x[0]/x[1],2))) \
    .map(lambda x : (x[0], (x[1]), (dict_meta_bd.value[x[0]]))) \
    .map(lambda x : (
        x[0],
        x[1],
        re.sub(r'\((\d+)\)', "" ,x[2][0] ).lower().strip(),
        int(re.findall(r'\((\d+)\)' , x[2][0])[0]), 
        x[2][1].lower().split("|")
        )
    ) \
    .sortBy(lambda x : -x[1])
    


use_rat.collect()

[('3280', 5.0, 'baby, the', 1973, 'horror'),
 ('989', 5.0, 'schlafes bruder (brother of sleep)', 1995, 'drama'),
 ('1830', 5.0, 'follow the bitch', 1998, 'comedy'),
 ('3656', 5.0, 'lured', 1947, 'crime'),
 ('787', 5.0, 'gate of heavenly peace, the', 1995, 'documentary'),
 ('3881', 5.0, 'bittersweet motel', 2000, 'documentary'),
 ('3233', 5.0, 'smashing time', 1967, 'comedy'),
 ('3172', 5.0, 'ulysses (ulisse)', 1954, 'adventure'),
 ('3382', 5.0, 'song of freedom', 1936, 'drama'),
 ('3607', 5.0, 'one little indian', 1973, 'comedy|drama|western'),
 ('3245', 4.8, 'i am cuba (soy cuba/ya kuba)', 1964, 'drama'),
 ('53', 4.75, 'lamerica', 1994, 'drama'),
 ('2503', 4.67, 'apple, the (sib)', 1998, 'drama'),
 ('2905', 4.61, 'sanjuro', 1962, 'action|adventure'),
 ('2019',
  4.56,
  'seven samurai (the magnificent seven) (shichinin no samurai)',
  1954,
  'action|drama'),
 ('318', 4.55, 'shawshank redemption, the', 1994, 'drama'),
 ('745', 4.52, 'close shave, a', 1995, 'animation|comedy|thriller')

In [35]:
use_rat = sc.textFile(r"ml-1m\ratings.dat") \
    .map(lambda x: x.split("::")) \
    .map(lambda x: [x[1], int(x[2]), x[0]]) \
    .map(lambda x : (x[0],(x[1],x[2]), dict_users_bd.value[x[-1]]))

use_rat.take(10)

[('1193', (5, '1'), ('F', 1)),
 ('661', (3, '1'), ('F', 1)),
 ('914', (3, '1'), ('F', 1)),
 ('3408', (4, '1'), ('F', 1)),
 ('2355', (5, '1'), ('F', 1)),
 ('1197', (3, '1'), ('F', 1)),
 ('1287', (5, '1'), ('F', 1)),
 ('2804', (5, '1'), ('F', 1)),
 ('594', (4, '1'), ('F', 1)),
 ('919', (4, '1'), ('F', 1))]

In [ ]:
Endpoint,Argumetos,Ejemplos
RATE_TOP20,year:int,genre:str,http://localhost:5000/RATE_TOP20?year=2000 | http://localhost:5000/RATE_TOP20?genre=drama | http://localhost:5000/RATE_TOP20?genre=drama&year=1993
RATE_BOTTOM20,year:int,genre:str,http://localhost:5000/RATE_BOTTOM20?year=2000 | http://localhost:5000/RATE_BOTTOM20?genre=drama | http://localhost:5000/RATE_BOTTOM20?genre=drama&year=1993
COUNT_TOP20,year:int,genre:str,http://localhost:5000/COUNT_TOP20?year=2000 | http://localhost:5000/COUNT_TOP20?genre=drama | http://localhost:5000/COUNT_TOP20?genre=drama&year=1993
MOVIE,movie:str,http://localhost:5000/MOVIE?movie=american beauty
LISTBYGENDER,gender:str,http://localhost:5000/LISTBYGENDER?gender=comedy
SUGGEST,genero:str,edad:int,localhost:5000/SUGGEST?genero=F | http://localhost:5000/SUGGEST?edad=43 | http://localhost:5000/SUGGEST?edad=43&genero=M
